# Introduction
Le but de ce notebook est de relever les anomalies présentes sur les notices d'exemplaire afin de pouvoir exporter des listes à destination des aquéreurs. Le script se compose des parties suivantes :
1. La visualisation de la table des notices d'exemplaire
2. Une suite de requêtes sur concernant les différentes colonnes de notre dataframe nommé `items`

# 1.Visualisation de la table des notices d'exemplaire

In [62]:
import pandas as pd
from datetime import datetime

from kiblib.utils.db import DbConn

In [63]:
db_conn = DbConn().create_engine()

On définit la variable `query` comme une requête SQL dans laquelle aux champs de la table `items` on a ajouté le champs `itemtype` de la table `biblioitems`:

In [64]:
query = """SELECT i.itemnumber, i.biblionumber, i.biblioitemnumber, i.barcode, i.dateaccessioned, i.booksellerid, i.homebranch, i.price, i.replacementprice, i.replacementpricedate, i.datelastborrowed, i.datelastseen, i.stack, i.notforloan, i.damaged, i.damaged_on, i.itemlost, i.itemlost_on, i.withdrawn, i.withdrawn_on, i.itemcallnumber, i.coded_location_qualifier, i.issues, i.renewals, i.reserves, i.restricted, i.itemnotes, i.itemnotes_nonpublic, i.holdingbranch,i.timestamp, i.location, i.permanent_location, i.onloan, i.cn_source, i.cn_sort, i.ccode, i.materials, i.uri, i.itype, i.more_subfields_xml, i.enumchron, i.copynumber, i.stocknumber, i.new_status, i.exclude_from_local_holds_priority, bi.itemtype
FROM koha_prod.items i
JOIN koha_prod.biblioitems bi ON bi.biblionumber = i.biblionumber """

In [66]:
items = pd.read_sql(query,db_conn)
items['damaged'] = items['damaged'].astype(str)

> Par convention, la requête SQL est enfermée entre `"""..."""`

On définit ensuite la variable `items` qui se construit à partir de la fonction `pd.read_sql()` et de la variable `query` puis l'on affiche la variable `items`:

## Transformation des valeurs autorisées en colonnes compréhensibles

In [5]:
query2 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'damaged'"""

In [6]:
damaged_va = pd.read_sql(query2, db_conn)
damaged_va['authorised_value'] = damaged_va['authorised_value'].astype(str)

In [7]:
items_damaged_va = items.merge(damaged_va, left_on="damaged", right_on="authorised_value")
items_damaged_va = items_damaged_va.rename(columns={"authorised_value": "valeurs autorisées", "lib": "endommagé"})
items_damaged_va

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype,valeurs autorisées,endommagé
0,1,1,1,C0001353993,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
1,3,1,1,C0000653853,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
2,4,1,1,C0003476991,2005-03-22,None,MED,1.00,1.00,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
3,5,1,1,C0001499529,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
4,8,1,1,C0001353935,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287954,429463,335235,335235,C1400010558,2021-07-08,None,MED,20.00,NaN,2021-07-08,...,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,LI,2,Prétendu rendu
287955,425062,335878,335878,C2500022405,2021-04-02,None,MED,12.00,NaN,2021-04-02,...,PRETLIV,None,None,None,None,None,NaN,LI,2,Prétendu rendu
287956,429884,338914,338914,C2500023806,2021-07-16,None,MED,10.50,NaN,2021-07-16,...,PRETLIV,None,None,None,None,None,NaN,LI,2,Prétendu rendu
287957,434392,340475,340475,C2500025144,2021-10-30,None,MED,NaN,NaN,2021-10-30,...,PRETLIV,None,None,None,None,None,NaN,LI,2,Prétendu rendu


# 2. Recherche des anomalies 

Les listes d'anomalies vont être envoyées aux collègues. Dans un souci de lisibilité, on souhaite réduire aux nombre de colonnes essentielles pour les acquéreurs. On va donc dans un premier temps **afficher l'ensemble des colonnes** de notre dataframe :

On va ensuite définir la variable `colonnes_a_exporter` contenant la liste des colonnes que l'on souhaite sélectionner pour la génération des listes : 

In [8]:
colonnes_a_exporter = ['barcode',
       'dateaccessioned', 'homebranch', 'price',
       'replacementprice', 'datelastborrowed',
       'datelastseen', 'notforloan', 'damaged', 'damaged_on',
       'itemlost', 'itemlost_on', 'withdrawn', 'withdrawn_on',
       'itemcallnumber','holdingbranch', 'timestamp', 'location',
       'onloan', 'ccode','itemnotes','itemnotes_nonpublic','itemtype']

In [9]:
colonnes_a_exporter_damaged_va = ['barcode','endommagé',
       'dateaccessioned', 'homebranch', 'price',
       'replacementprice', 'datelastborrowed',
       'datelastseen', 'notforloan', 'damaged', 'damaged_on',
       'itemlost', 'itemlost_on', 'withdrawn', 'withdrawn_on',
       'itemcallnumber','holdingbranch', 'timestamp', 'location',
       'onloan', 'ccode','itemnotes','itemnotes_nonpublic','itemtype']

## Anomalie sur le type de document (niveau notice) `itemtype`
On souhaite ici générer une liste des document dont le `itemtype`n'est pas renseigné :

In [58]:
anomalie_type_document = items[items['itemtype'].isna()]
anomalie_type_document

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,materials,uri,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype
280041,437683,343936,343936,C1300000515,2021-12-30,None,MED,9.00,NaN,2021-12-30,...,None,None,PRETLIV,None,None,None,None,None,NaN,None
280540,439787,344427,344427,C2500026686,2022-02-15,None,MED,7.10,7.1,2022-02-15,...,None,None,PRETLIV,None,None,None,None,None,NaN,None
280796,440009,344663,344663,C3100010261,2022-02-22,None,MED,19.95,NaN,2022-02-22,...,None,None,PRETSON,None,None,None,None,None,NaN,None
280806,439990,344673,344673,C3100010257,2022-02-22,None,MED,19.95,NaN,2022-02-22,...,None,None,PRETSON,None,None,None,None,None,NaN,None
280807,439991,344673,344673,C3100010256,2022-02-22,None,MED,19.95,NaN,2022-02-22,...,None,None,PRETSON,None,None,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287945,452446,351404,351404,C1400015030,2022-11-15,None,MED,NaN,NaN,2022-11-15,...,None,None,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,None
287946,452445,351405,351405,C1400015031,2022-11-15,None,MED,5.90,NaN,2022-11-15,...,None,None,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,None
287947,452447,351406,351406,C1400015029,2022-11-15,None,MED,NaN,NaN,2022-11-15,...,None,None,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,None
287948,452455,351407,351407,C1400015024,2022-11-15,None,MED,23.90,NaN,2022-11-15,...,None,None,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,None


In [11]:
anomalie_type_document[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_typedoc1.xlsx',index=False)

## Anomalies sur les codes-barre

On définit ici la variable `barcode` comme liste des notices pour laquelle il n'y a pas de code-barres grâce à la fonction `.isna()`:

In [59]:
barcode_ko = items[items['barcode'].isna()]
barcode_ko

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,materials,uri,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype
491,451926,910,910,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
45221,430326,70043,70043,None,2021-07-27,None,MED,NaN,NaN,2021-07-27,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
55377,432377,84901,84901,None,2021-09-18,None,MED,NaN,NaN,2021-09-18,...,None,None,PRETLIV,None,None,None,None,None,NaN,PA
70347,439857,103787,103787,None,2022-02-18,None,BUS,4.8,4.8,2022-02-18,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
88208,142053,125932,125932,None,2005-03-25,None,MED,40.0,40.0,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287898,451962,351331,351331,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
287899,451964,351332,351332,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
287900,451965,351333,351333,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
287901,451966,351334,351334,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI


Une notice d'exemplaire peut ne pas posséder de code-barres pour diverses raisons. Il nous faut checker leur statut de prêt. Afin d'avoir une vue d'ensemble on va compter le nombre notices d'exemplaires par statut de prêt (notforloan) :

In [13]:
barcode_ko['notforloan'].value_counts()

-1    373
-2     18
 0     14
-4      4
-3      2
Name: notforloan, dtype: int64

Pour comprendre à quoi correspondent les valeurs ci-dessus, il faut se référer à la liste des valeurs autorisées :

| Valeurs Autorisées| État 
|-------------------|------
|-4| En répération
|-3| En retrait
|-2| En traitement
|-1| En commande
|0 | Empruntable
|2 | Réservé à la consultation sur place
|3 | Consultable au musée
|4 | Sorti des collections
|5 | En reliure

Sont définis commes des anomalies de code-barre, les documents dont le statut est :
* empruntable
* en répération
* en retrait

On définit donc la variabe qui suit que l'on exporte ensuite : 

In [14]:
anomalies_barcode_ko = barcode_ko[barcode_ko['notforloan'].isin([0,-4,-3])]

In [15]:
anomalies_barcode_ko[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_code_barres.xlsx',index=False)

> Il n'existe pas de fonction `.isnotin`
* Si l'on souhaite faire une sélection qui excluent certaines valeurs, la syntaxe est un peu particulière. On garde la fonction `.isin` et on ajoute un `~` devant le nom de la colonne qui nous intéresse. Ce qui donne dans notre exemple : `barcode[~barcode['notforloan'].isin([-1,-2])]` 

# Anomalies sur la localisation des documents
On souhaite générer une liste des notices d'exemplaire dont le site de rattachement (`homebranch`) n'est pas renseigné :

In [16]:
items[items['homebranch'].isna()]

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,materials,uri,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype


Idem pour la localisation à l'intérieur la médiathèque (`location`) :

In [1]:
location_ko = items[items['location'].isna()]
location_ko

NameError: name 'items' is not defined

Les documents peuvent ne pas avoir de localisation interne renseignée pour deux raisons : 
* document en commande
* document en traitement

En dehors de ces états de prêt (`notforloan`) les documents dont la localisation n'est pas renseignée est considéré comme une anomalie, définie ici par la variable `anomalies_location_ko` de la manière suivante : 

In [18]:
anomalies_location_ko = location_ko[~location_ko['notforloan'].isin([-2,-1])]

* [x] Trier les docs par statut comme pour les codes-barre

In [19]:
anomalies_location_ko[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_location_ko.xlsx',index=False)

### Les localisations incohérentes
On souhaite ensuite générer une liste des notices d'exemplaire dont le site de rattachement et le site de retrait des réservations est incohérent :
* Pour le bus
* Pour la médiathèque

In [20]:
anomalies_loc_incoherente_bus = items[(items["homebranch"] == "BUS") & (items["location"] != "BUS1A")]

In [21]:
anomalies_loc_incoherente_med = items[(items['homebranch'] == 'MED') & (items['location'].str[0:3] != 'MED')&(~items['location'].isna())]

In [22]:
anomalies_loc_incoherente_med[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_loc_incoherente_med.xlsx',index=False)

In [23]:
anomalies_loc_incoherente_bus[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_loc_incoherente_bus.xlsx',index=False)

# Anomalie sur les prix d'acquisition
On souhaite générer une liste des notices pour lesquelles le prix n'est pas renseigné (NaN)

In [24]:
prix_ko = items[items['price'].isna()]
prix_ko

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,materials,uri,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype
55,155,68,68,C0003165265,2005-03-22,None,MED,NaN,NaN,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
71,191,104,104,C0002267705,2005-03-22,None,MED,NaN,NaN,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
72,192,104,104,C0003695812,2005-03-22,None,MED,NaN,NaN,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
123,332,243,243,C0000256457,2005-03-22,None,MED,NaN,NaN,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
140,376,283,283,C0000256520,2005-03-22,None,MED,NaN,NaN,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287954,452480,351555,351555,C3100010685,2022-11-16,None,MED,NaN,NaN,2022-11-16,...,None,None,PRETSON,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,DC
287955,452481,351556,351556,C3100010686,2022-11-16,None,MED,NaN,NaN,2022-11-16,...,None,None,PRETSON,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,DC
287956,452482,351557,351557,C3100010687,2022-11-16,None,MED,NaN,NaN,2022-11-16,...,None,None,PRETSON,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,DC
287957,452483,351558,351558,C1400015034,2022-11-16,None,MED,NaN,NaN,2022-11-16,...,None,None,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,LI


Les exemplaires dont le prix n'est pas renseigné ne sont pas tous à considérer commes des anomalies. Pour les documents patrimoniaux, c'est normal. On définit donc une variable `anomalies_prix_ko` qui exclue les documents du patrimoine d'abord :

In [25]:
anomalies_prix_ko = prix_ko[(prix_ko['ccode'].str[0] != 'P')]

In [26]:
anomalies_prix_ko = anomalies_prix_ko[anomalies_prix_ko['itemtype']!='PE'] # On exlue par cette syntaxe les périodiques.

On transforme ensuite le type des valeurs de la colonne date d'acquisition `dateaccessioned` qui sont des chaînes de caractères `string`, en dates : 

In [27]:
anomalies_prix_ko['dateaccessioned'] = pd.to_datetime(anomalies_prix_ko['dateaccessioned'])

Puis l'on filtre à nouveau via la fonction `groupby` et `dt.year` et `.size` pour obtenir une liste des exemplaires considérés comme des anomalies et dont le prix n'est pas renseigné, par an : 

In [28]:
anomalies_prix_ko_by_dateaccessioned = anomalies_prix_ko.groupby([anomalies_prix_ko['dateaccessioned'].dt.year]).size()
anomalies_prix_ko_by_dateaccessioned

dateaccessioned
2005    4285
2006       3
2007       2
2008     189
2009     282
2010     604
2011     133
2012       1
2013       1
2014       1
2015       5
2016     192
2017     188
2018     114
2019     315
2020    1296
2021    3020
2022    3248
dtype: int64

On constate ici que le nombre d'exemplaires sans prix renseigné a fortement augmenté depuis 2020, on souhaite filtrer les anomalies depuis 2020 par localisation. On définit d'abord une variable `anomalies_prix_ko_ap_2020` et l'on définit à partir de cette première variable une variable par localisation `anomalies_prix_ko_ap_2020_by_loc` :

In [29]:
anomalies_prix_ko_by_loc = anomalies_prix_ko[anomalies_prix_ko['dateaccessioned'].dt.year >= 2020]
anomalies_prix_ko_by_loc = anomalies_prix_ko_by_loc.groupby(anomalies_prix_ko['location']).size()
anomalies_prix_ko_by_loc

location
BUS1A     314
MED0A    4422
MED0C      68
MED1A     721
MED2A     258
MED2B       7
MED2C       1
MED3A    1377
MED3C       9
MED3D       4
MED3E      25
MED3F       1
MED3I       7
MED3J       2
MED3K       1
MED3M       3
dtype: int64

In [30]:
anomalies_prix_ko_by_loc = anomalies_prix_ko[anomalies_prix_ko['location'] == 'MED1A']
anomalies_prix_ko_by_loc = anomalies_prix_ko_by_loc.groupby(anomalies_prix_ko['ccode']).size()
anomalies_prix_ko_by_loc

ccode
AAPATAP     8
AAPATLC     2
AAPCGDA     4
AAPCGVI    18
AAPCIDC     1
AAPCIFI    20
AAPCIMF    29
AAPCITV     4
AAPPLCB     7
AAPPLFF     3
AAPPLGO     4
AAPPLHU     3
AAPPLMS     8
AAPPLPS     2
AAPPLSL     5
ACFPAZZ    10
ALTDCZZ     3
ALTIGFA     9
ALTIGSF     5
ALTLSZZ    72
ALTLVZZ     3
ALTPOZZ     2
ALTROAD    15
ALTROAM    20
ALTROHI     3
ALTROPL    45
ALTROTE     1
ALTROZZ    55
ALTTCCN     1
ALTTCNV     3
ALTTLZZ     4
AMV00ZZ     1
AMV11ZZ    15
AMV13ZZ    35
AMV14ZZ    11
AMV15ZZ     2
AMV16ZZ     1
AMV20ZZ    39
AMV25ZZ    21
AMV29ZZ    12
AMV30ZZ    46
AMV39ZZ     9
AMV40ZZ     1
AMV50ZZ    30
AMV80ZZ    50
AMV90ZZ    89
AMVASDC     1
AMVTHDC     3
AMVTHTX    99
dtype: int64

In [31]:
anomalies_prix_ko_by_loc.to_excel('liste_anomalies_exemplaires_prix_ko_by_loc.xlsx')

* [ ] regarder par localisation (location)

In [32]:
anomalies_prix_ko[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_prix.xlsx',index=False)

# Anomalies sur les statuts d'exemplaires


In [33]:
items['notforloan'].value_counts()

 0    217921
 2     55505
 4      8946
-3      2009
-2      1282
-4      1222
 3       595
-1       373
 5       106
Name: notforloan, dtype: int64

In [34]:
deadline_1_an = pd.to_datetime("today") - pd.Timedelta(365, unit='D')
deadline_1_an

Timestamp('2021-11-17 09:27:28.334405')

## Documents abimés

In [35]:
items_damaged_va

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype,valeurs autorisées,endommagé
0,1,1,1,C0001353993,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
1,3,1,1,C0000653853,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
2,4,1,1,C0003476991,2005-03-22,None,MED,1.00,1.00,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
3,5,1,1,C0001499529,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
4,8,1,1,C0001353935,2005-03-22,None,MED,8.99,8.99,None,...,PRETLIV,None,None,None,None,None,NaN,LI,0,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287954,429463,335235,335235,C1400010558,2021-07-08,None,MED,20.00,NaN,2021-07-08,...,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,LI,2,Prétendu rendu
287955,425062,335878,335878,C2500022405,2021-04-02,None,MED,12.00,NaN,2021-04-02,...,PRETLIV,None,None,None,None,None,NaN,LI,2,Prétendu rendu
287956,429884,338914,338914,C2500023806,2021-07-16,None,MED,10.50,NaN,2021-07-16,...,PRETLIV,None,None,None,None,None,NaN,LI,2,Prétendu rendu
287957,434392,340475,340475,C2500025144,2021-10-30,None,MED,NaN,NaN,2021-10-30,...,PRETLIV,None,None,None,None,None,NaN,LI,2,Prétendu rendu


In [36]:
items_damaged_va['damaged'].value_counts()

0    284855
1      3044
2        60
Name: damaged, dtype: int64

In [37]:
docs_abimes = items_damaged_va[items_damaged_va['endommagé'] == 'Abimé']
docs_abimes

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype,valeurs autorisées,endommagé
284855,208,120,120,C0000236732,2005-03-22,None,MED,9.15,9.15,None,...,PRETLIV,None,None,None,None,None,NaN,LI,1,Abimé
284856,802,674,674,C0001377264,2005-03-22,None,MED,18.00,18.00,None,...,PRETSON,None,None,None,None,None,NaN,DC,1,Abimé
284857,1050,909,909,C0001165120,2005-03-22,None,MED,2.29,2.29,None,...,PRETLIV,None,None,None,None,None,NaN,LI,1,Abimé
284858,1051,910,910,C0001165269,2005-03-22,None,MED,9.15,9.15,None,...,PRETLIV,None,None,None,None,None,NaN,LI,1,Abimé
284859,1216,1069,1069,C0001178644,2005-03-22,None,MED,21.34,21.34,None,...,PRETLIV,None,None,None,None,None,NaN,LI,1,Abimé
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287894,446087,347978,347978,C2500028016,2022-06-30,None,BUS,NaN,10.90,2022-06-30,...,PRETLIV,None,None,None,None,None,NaN,LI,1,Abimé
287895,446823,348451,348451,C1400014172,2022-07-13,None,MED,8.40,NaN,2022-07-13,...,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,LI,1,Abimé
287896,448832,348956,348956,C2500028899,2022-09-07,None,MED,5.00,5.00,2022-09-07,...,PRETLIV,None,None,None,None,None,NaN,LI,1,Abimé
287897,448338,349136,349136,C1400014390,2022-08-25,None,MED,22.00,NaN,2022-08-25,...,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,LI,1,Abimé


On va ensuite convertir la colonne `'damaged_on'` qui est en `string` en valeur `date` :

In [38]:
items_damaged_va['damaged_on'] = pd.to_datetime(items_damaged_va['damaged_on'],format='%y%m%d')

On définit la variable `anomalies_abimes`, une liste des `docs_abimes` et dont le statut `damaged_on` remonte à une date supérieure à notre deadline (une année) :

In [39]:
anomalies_abimes = docs_abimes[docs_abimes['damaged_on'] < deadline_1_an]
anomalies_abimes['damaged_on']

284862   2021-01-05 10:57:08
284872   2020-12-22 15:19:05
284873   2021-09-18 15:58:41
284885   2020-08-19 14:09:37
284889   2021-01-20 13:20:52
                 ...        
287779   2021-09-14 09:35:19
287785   2021-06-10 16:17:11
287792   2021-08-14 13:46:28
287807   2021-11-03 12:54:13
287826   2021-10-30 09:38:28
Name: damaged_on, Length: 434, dtype: datetime64[ns]

In [40]:
anomalies_abimes.groupby('location').size()

location
BUS1A     25
MED0A    132
MED0C     10
MED1A     92
MED2A    153
MED2C      2
MED3A     15
MED3C      1
MED3E      1
MED3F      1
MED3H      1
dtype: int64

In [41]:
anomalies_abimes[colonnes_a_exporter_damaged_va].to_excel('liste_anomalies_docs_abimes.xlsx',index=False)

Si l'on souhaite mettre un chemin en mode chemin relatif (cheminement à rebours) on utilisera `../data/liste_anomalies_exemplaires_abimes1.xlsx` etc...

## Documents en retrait
Sont considérés commes des anomalies de retrait les documents en statut retrait depuis plus d'un an

In [42]:
items['timestamp'] = pd.to_datetime(items['timestamp'],format='%y%m%d')

In [43]:
docs_en_retrait = items[items['notforloan'] == -3]
docs_en_retrait

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,materials,uri,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype
513,1077,933,933,C0002156417,2005-03-22,None,MED,19.06,19.06,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
925,1865,1684,1684,C0001382196,2005-03-22,None,MED,18.00,18.00,None,...,None,None,PRETSON,None,None,None,None,None,NaN,DC
1568,2897,2622,2622,C0001381302,2005-03-22,None,MED,18.00,18.00,None,...,None,None,PRETSON,None,None,None,None,None,NaN,DC
2343,4181,3779,3779,C0001450614,2005-03-22,None,MED,24.00,24.00,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
2803,4896,4376,4376,C0001648348,2005-03-22,None,MED,55.45,55.45,None,...,None,None,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,LI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286640,450115,350070,350070,C2500029323,2022-10-06,None,MED,13.00,13.00,2022-10-06,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
286672,449887,350094,350094,C2500029327,2022-10-04,None,MED,13.00,13.00,2022-10-04,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
286674,450272,350095,350095,C2500029416,2022-10-07,None,MED,12.50,12.50,2022-10-07,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
286685,449888,350103,350103,C2500029328,2022-10-04,None,MED,9.50,9.50,2022-10-04,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI


In [44]:
anomalies_docs_en_retrait = docs_en_retrait[docs_en_retrait['timestamp'] < deadline_1_an]
anomalies_docs_en_retrait.groupby('location').size()

location
BUS1A      3
MED0A    243
MED0C     19
MED1A    152
MED2A    110
MED2C     24
MED3A     38
MED3C      3
MED3E     39
MED3F      6
MED3G    112
MED3H     25
MED3I      9
MED3J      2
MED3K      2
MUS1A     53
dtype: int64

> * [ ] En l'état actuel, on a une liste des docs en retrait depuis + d'1 an. Voir si l'on peut affiner.

In [45]:
anomalies_docs_en_retrait[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_docs_en_retrait.xlsx',index=False)

## Documents en commande
On génère une liste des documents en commande :

In [46]:
docs_en_commande = items[items['notforloan'] == -1]
docs_en_commande

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,materials,uri,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype
491,451926,910,910,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
45221,430326,70043,70043,None,2021-07-27,None,MED,NaN,NaN,2021-07-27,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
114976,416328,160474,160474,None,2020-10-28,None,MED,NaN,NaN,2020-10-28,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
122213,451883,169537,169537,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
131785,451900,182292,182292,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287898,451962,351331,351331,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
287899,451964,351332,351332,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
287900,451965,351333,351333,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
287901,451966,351334,351334,None,2022-11-05,None,MED,NaN,NaN,2022-11-05,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI


In [47]:
anomalies_docs_en_commande = docs_en_retrait[docs_en_retrait['timestamp'] < deadline_1_an]
anomalies_docs_en_commande

,itemnumber,biblionumber,biblioitemnumber,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,replacementpricedate,...,materials,uri,itype,more_subfields_xml,enumchron,copynumber,stocknumber,new_status,exclude_from_local_holds_priority,itemtype
513,1077,933,933,C0002156417,2005-03-22,None,MED,19.06,19.06,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
2343,4181,3779,3779,C0001450614,2005-03-22,None,MED,24.00,24.00,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
2803,4896,4376,4376,C0001648348,2005-03-22,None,MED,55.45,55.45,None,...,None,None,PRETLIV,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<colle...",None,None,None,None,NaN,LI
2896,5024,4497,4497,C0002002503,2005-03-22,None,MED,10.50,10.50,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
3734,6236,5605,5605,C0001640007,2005-03-22,None,MED,0.15,0.15,None,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259028,407398,324776,324776,C2500017309,2020-03-10,None,MED,6.95,NaN,2020-03-10,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
261236,410048,326775,326775,C2500018095,2020-06-27,None,MED,12.50,NaN,2020-06-27,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
261269,410050,326803,326803,C2500018105,2020-06-27,None,MED,12.90,NaN,2020-06-27,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI
271338,425460,336223,336223,C2500022596,2021-04-15,None,MED,6.95,6.95,2021-04-15,...,None,None,PRETLIV,None,None,None,None,None,NaN,LI


In [48]:
anomalies_docs_en_commande.groupby(items['location']).size()

location
BUS1A      3
MED0A    243
MED0C     19
MED1A    152
MED2A    110
MED2C     24
MED3A     38
MED3C      3
MED3E     39
MED3F      6
MED3G    112
MED3H     25
MED3I      9
MED3J      2
MED3K      2
MUS1A     53
dtype: int64

On trouve des docs en commande depuis plus d'u an majoritairement : 
* Au RDC
* Au Lac
* En jeunesse ?
* [ ] Voir si possibilité d'affiner

In [49]:
anomalies_docs_en_commande[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_docs_en_commande.xlsx',index=False)

# Anomalies sur les côtes de collection

On souhaite afficher la liste des exemplaires sans côte renseignée. On définit donc la variable `cote_ko`. Sont ensuite considérés comme des anomalies les exemplaires sans côte qui ne sont ni en commande, ni en traitement :

In [50]:
cotes_ko = items[items['cn_sort'].isna()]
anomalies_cotes_ko = cotes_ko[~cotes_ko['notforloan'].isin([-2,-1])]

In [51]:
anomalies_cotes_ko[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_docs_cotes_ko.xlsx',index=False)

# Anomalies sur les codes de collection

In [52]:
code_collection_ko = items[items['ccode'].isna()]
code_collection_ko.groupby('notforloan').size()

notforloan
-4      5
-3      3
-2     14
-1    334
 0    235
 2     34
 4      1
dtype: int64

In [53]:
anomalies_code_collection_ko = code_collection_ko[~code_collection_ko['notforloan'].isin([-2,-1])] # docs ni en commande, ni en traitement
anomalies_code_collection_ko.groupby(['notforloan','location','itemtype']).size() #Docs sans ccode réparti par Statut, localisation & type de doc

notforloan  location  itemtype
-4          MED3H     LI            1
-3          MED1A     VD            1
            MED2A     LI            1
 0          BUS1A     LI            6
            MED0A     LI            5
            MED0C     LI           16
                      VD            1
            MED1A     DC            9
                      DG            2
                      DV          124
                      LG            1
                      LI           12
                      PA            1
                      VD           19
            MED2A     LI           14
                      PE            1
            MED3A     LI            7
            MED3C     DC            4
                      DV            6
            MED3E     LI            1
 2          MED2A     LI            1
            MED3C     DV            1
            MED3I     LI           16
            MED3J     LI            2
            MED3K     LI            1
            MED3M  

In [54]:
anomalies_code_collection_ko[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_collection_code_ko.xlsx',index=False)

#  Anomalies sur les notes de notices

On souhaite ici générer une liste des notices d'exemplaire qui possèdent une note

In [55]:
anomalies_notes = items[(~items['itemnotes'].isna()) | (~items['itemnotes_nonpublic'].isna())]

In [56]:
anomalies_notes[colonnes_a_exporter].to_excel('liste_anomalies_exemplaires_notes.xlsx',index=False)

> On pourrait considérer que après 1 an, les notes sur les notices d'exemplaire peuvent être supprimées ? (demander à F.)
- - - - -

# Choses à faire
* [x] Regarder les documents en retrait
* [x] Regarder les documents en commande
* [x] Regarder les docs sans côte
* [ ] Regarder les docs sans côte de collection
* [ ] Pour les docs en DAMAGED_ON sortir une liste avec les valeurs dans les colonnes de VA remplacés par leur libelé (regarder manipulation faite par françois)